In [1]:
import pandas as pd
import geopandas as gpd
import fnmatch
import os
import datetime
from datetime import timedelta
import glob

In [2]:
os.environ['PROJ_LIB'] = 'C:/ProgramData/Miniconda3/Library/share/proj'
os.environ['GDAL_DATA'] = 'C:/ProgramData/Miniconda3/Library/share'
pyproj_datadir="C:\Anaconda3\Library\share"

In [3]:
sd = gpd.read_file('C:/DATA/aus_star_transects/NT_StarTransect_20211111_sj.shp')
sd.crs

{'init': 'epsg:4326'}

In [7]:
sd6 = gpd.read_file('test7.shp')
sd6.crs

{}

In [4]:
sd['date_time'] = sd.obs_time.apply(pd.to_datetime)

In [5]:
print (sd.WRSPR.unique())

['105070' '106069' '102073' '105072' '103076' '104075' '103071' '103072'
 '102076' '100077' '099078' '101074' '106071' '106072' '107070' '106073'
 '106070' '105073' '105074' '105075' '105076' '105071' '106068' '104076'
 '105069' '103078' '103077' '104072' '104071' '103075' '104070' '102078'
 '102077' '103073' '102075' '103074' '104069' '103070' '104068' '102074'
 '102072' '101078' '101077' '102071' '101076' '101075' '101073' '101072'
 '100076' '100075' '101071' '100074' '100073' '100072']


In [6]:
print (list(sd))

['Join_Count', 'TARGET_FID', 'publish', 'site', 'obs_time', 'longitude', 'latitude', 'persist', 'crust', 'dist', 'rock', 'green', 'dead', 'litter', 'crypto', 'mid_g', 'mid_d', 'mid_b', 'crn', 'over_g', 'over_d', 'over_b', 'num_points', 'unoccluded', 'obs_key', 'Field24', 'Field25', 'fpc', 'AREA', 'PERIMETER', 'PR_', 'PR_ID', 'RINGS_OK', 'RINGS_NOK', 'PATH', 'ROW', 'MODE', 'SEQUENCE', 'WRSPR', 'PR', 'ACQDayL7', 'ACQDayL8', 'geometry', 'date_time']


In [ ]:
sd["fwd_date"] = sd["date_time"] + timedelta(days=30)
sd["bck_date"] = sd["date_time"] + timedelta(days=-30)

In [ ]:
%run C:\Users\grants\code\file_management\list_of_files_multi_dir.py -d Z:/Landsat/wrs2/ -e *dilm[2-3]_zstdmask.img -o test_imglist_dev.csv

In [ ]:
df = pd.read_csv('test_imglist_dev.csv',header=None)
# get the path row and image date from the image file name
df['path_row'] = df[0].map(lambda x: str(x)[-35:-32]) + '_' + df[0].map(lambda x: str(x)[-31:-28])
df['img_date'] = df[0].map(lambda x: str(x)[-27:-23]) + '-' + df[0].map(lambda x: str(x)[-23:-21]) + '-' + df[0].map(lambda x: str(x)[-21:-19])
df['zone'] = df[0].map(lambda x: str(x)[-14:-13])
print (df['zone'].unique())
df.to_csv('imglist_proc.csv')


#for index, row in df.iterrows():
    #print (row[0],row['img_dt'])

In [ ]:
df = pd.read_csv('imglist_proc.csv',header=0)

print (list(df))

In [ ]:
# read in the shape file and produce a date_time 
sd = gpd.read_file('C:/DATA/aus_star_transects/NT_StarTransect_20211111_sj.shp')
sd['date_time'] = sd.obs_time.apply(pd.to_datetime)


# define the number of days either side of the field site measured date to extract stats from the available imagery 
date_plus = 40 
date_minus = -40
# create the plus and minus date range 
sd["fwd_date"] = sd["date_time"] + timedelta(days=date_plus)
sd["bck_date"] = sd["date_time"] + timedelta(days=date_minus)


# iterrate over the sites in the shapefile and find the image matches and extract out the statistics 

for index, row in sd.iterrows():
    
    site_zonal = row
    #site_zonal.to_file("temp_shp.shp")
    #print (site_zonal.crs)
    
    site = row['site']
    field_date = row['date_time']
    
    # get the date range for the imagery stats requred
    search_date_plus = row['fwd_date']
    search_date_minus = row['bck_date']
    
    # get the info to create the dir path to get a list of imagery 
    path_row = str(row['PATH']) + '_0' + str(row['ROW'])
    year = str(row['obs_time'][:4])
    
    #search the img list and find matches
    df = pd.read_csv('imglist_proc.csv',header=0)
    # make the img_date a time date column 
    df['img_dt'] = df.img_date.apply(pd.to_datetime)
        
    dfs = df[(df['path_row'] == path_row)]
    
    
    
    imgS = dfs[dfs["img_dt"].isin(pd.date_range(search_date_minus,search_date_plus))]
    
    zone = imgS['zone'].unique()
    print (zone)
    
    #if zone == 2:
        #site_zonal.crs 
    
    #print (img_selection.dtypes)
        
    for row in imgS:
        
        print ('Site name:',site)
        print ('field date:',field_date)
        print (path_row)
        print ('start search date:',search_date_minus)    
    
        print (imgS[['img_date','0','zone']])    
        print ('end search date:',search_date_plus)
    
    
        print ("...................")
    
    # select out the individual site and reproject it to projected coords UTM 52 or 53 and extract out the stats from 
    # the given img_selection list using the script - zonal_stats_single_perc.py
    
    # use geopandas to create the shp file in memory - may need to write it out as a file? if so make it temp and delete...
    
    # create a temp dir to save the individual results before concating them together 
    
   
       
    


In [ ]:
df['geometry'] = [Point(xy) for xy in zip(df['Long'], df['Lat'])]
# create a df with all of the attributes 
df2 = df[['geometry','Collectors','Date','Lat', 'Long','photo','Primary','Secondary','Tertiary','lu_code','lu_coden','Commodity','Variety','Management','Confidence','Comments','UIDkey']]
# create the geo data frame        
df2 = geopandas.GeoDataFrame(df2, geometry='geometry')
# set the projection and datum
df2.crs = "+init=epsg:4326"
# save out the geo data frame in esri shapefile format.
df2.to_file(shapeName, driver='ESRI Shapefile')  

In [ ]:
sds.set_geometry ?

In [6]:
sd = gpd.read_file('C:/DATA/aus_star_transects/NT_StarTransect_20211111_sj.shp')
#sd['date_time'] = sd.obs_time.apply(pd.to_datetime)

print (list(sd))

sda = sd[(sd['site'] == 'mgp03' )]
print (sda)

#sds = sda[['Join_Count', 'TARGET_FID', 'publish', 'site', 'obs_time', 'longitude', 'latitude', 'persist', 'crust', 
#          'dist', 'rock', 'green', 'dead', 'litter', 'crypto', 'mid_g', 'mid_d', 'mid_b', 'crn', 'over_g', 'over_d', 
#          'over_b', 'num_points', 'unoccluded', 'obs_key', 'fpc', 'WRSPR',  'geometry']]

sds = gpd.GeoDataFrame(sda, geometry=sda['geometry'])
sds.crs  #= "+init=epsg:4326" 
print (sds.crs)
#epsg_code_proj4 = '+proj=utm +zone=52 +south +ellps=WGS84 +datum=WGS84 +units=m +no_defs'

sdsr = sds.to_crs({'init': 'epsg:32752'})  # ("EPSG:32752")      #
sdsr.crs 
print (sdsr.crs)
sdsr.to_file('test7.shp', driver='ESRI Shapefile')  

#print (sda)



#df['geometry'] = [Point(xy) for xy in zip(df['Long'], df['Lat'])]

['Join_Count', 'TARGET_FID', 'publish', 'site', 'obs_time', 'longitude', 'latitude', 'persist', 'crust', 'dist', 'rock', 'green', 'dead', 'litter', 'crypto', 'mid_g', 'mid_d', 'mid_b', 'crn', 'over_g', 'over_d', 'over_b', 'num_points', 'unoccluded', 'obs_key', 'Field24', 'Field25', 'fpc', 'AREA', 'PERIMETER', 'PR_', 'PR_ID', 'RINGS_OK', 'RINGS_NOK', 'PATH', 'ROW', 'MODE', 'SEQUENCE', 'WRSPR', 'PR', 'ACQDayL7', 'ACQDayL8', 'geometry']
     Join_Count  TARGET_FID publish   site    obs_time   longitude   latitude  \
389           1         389       y  mgp03  2013-05-03  131.711319 -25.910403   

     persist  crust  dist  ...  RINGS_NOK  PATH  ROW  MODE  SEQUENCE   WRSPR  \
389    3.333   46.2   0.0  ...          0   103   78     D     25374  103078   

         PR  ACQDayL7  ACQDayL8                     geometry  
389  103078         8        16  POINT (131.71132 -25.91040)  

[1 rows x 43 columns]
{'init': 'epsg:4326'}
{'init': 'epsg:32752'}


C:\ProgramData\Anaconda3\envs\raster\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
    if epsg_code == 32752:
        epsg_code_proj4 = '+proj=utm +zone=52 +south +ellps=WGS84 +datum=WGS84 +units=m +no_defs'
    
    if epsg_code == 32753:
        epsg_code_proj4 = '+proj=utm +zone=53 +south +ellps=WGS84 +datum=WGS84 +units=m +no_defs'

In [23]:
ax = sdsr.plot()

In [ ]:
%run C:\Users\gwsta\code\zonal\zonal_stats_single_perc.py -h

In [ ]:
http://localhost:8888/edit/code/zonal/zonal_stats_single_perc.py

In [ ]:
    df = pd.read_csv(csvName, header=0)
    
    df['geometry'] = [Point(xy) for xy in zip(df['Long'], df['Lat'])]
    # create a df with all of the attributes 
    df2 = df[['geometry','Collectors','Date','Lat', 'Long','photo','Primary','Secondary','Tertiary','lu_code','lu_coden','Commodity','Variety','Management','Confidence','Comments','UIDkey']]
    # create the geo data frame        
    df2 = geopandas.GeoDataFrame(df2, geometry='geometry')
    # set the projection and datum
    df2.crs = "+init=epsg:4326"
    # save out the geo data frame in esri shapefile format.
    df2.to_file(shapeName, driver='ESRI Shapefile') 

In [ ]:
site_list=[]
site_date=[]


#for pathrow in sd.WRSPR.unique():
    
for index, row in sd.iterrows():
    
    site = row['site']
    field_date = row['date_time']
    
    # get the date range for the imagery stats requred
    search_date_plus = row['fwd_date']
    search_date_minus = row['bck_date']
    
    # get the info to create the dir path to get a list of imagery 
    path_row = str(row['PATH']) + '_0' + str(row['ROW'])
    year = str(row['obs_time'][:4])
    
    #month = row['obs_time'][5:7]
    
    print (path_row)
    
    print (search_date_minus)
    
    print (field_date)
    
    print (year)
        
    print (search_date_plus)
    
    print ("...................")
    
    dirname = 'C:/data/landsat/' + path_row + '/' + year + '/'
    
    print (dirname)
    print ("...................")
    
    pattern = '_zstdmask.img'
       
    
    img_list = []
    
    
    for root, dirs, files in os.walk(dirname):
        for file in files:
            if file.endswith(pattern):
                img = (os.path.join(root, file))
                img_list.append(img)
                #print (img)
    
    print (img_list)
    
    df = pd.DataFrame(img_list)
    
    

    #for img in img_list:
        
        #date = img[-27:-19]
        
        #if date > search_date_mimus & < search_date_minus: 
             
          

    

In [ ]:
dirname = 'D:/landsat/105_070/2011/'
    
print (dirname)
print ("...................")
    
pattern = '_zstdmask.img'
       
    
list_img = []
    
    
for root, dirs, files in os.walk(dirname):
    for file in files:
        if file.endswith(pattern):
            img = (os.path.join(root, file))
            list_img.append(img)
            print (img)
    
print (imglist)

In [ ]:
site_list=[]
site_date=[]


#for pathrow in sd.WRSPR.unique():
    
for index, row in sd.iterrows():
    
    site = row['site']
    date = row['date_time']
    year = row['obs_time'][:4]
    month = row['obs_time'][5:7]
    
    month_m1 = int(month)-1
    month_p1 = int(month)+1
    
    print (month_m1)
    print (month_p1)
    
    if month_p1 <=9:
        
        month_p1 = str(month_p1)[0:3] + '0' + str(month_p1)
        print ("...................")
        print (month_p1)
        print ("...................")
    if month_m1 <=9:   
        month_m1 = str(month_m1)[0:3] + '0' + str(month_m1)
        
    yearmonth = str(row['obs_time'][:4])+str(row['obs_time'][5:7])
    
    yearmonth_p1 = str(row['obs_time'][:4]+str(month_p1))
    yearmonth_m1 = str(row['obs_time'][:4]+str(month_m1))
    
    path_row = str(row['PATH']) + '_0' + str(row['ROW'])
    
    #print (site, ' ', date, ' ', year, ' ',month, ' ', path_row)
    print (site, ' ', date) 
    print ('plus month ', yearmonth_p1)
    print ('minus month ', yearmonth_m1)
    # select the folders to create the image list for each site
    
    # construct the file path from the site information 
    seach_month = [yearmonth_m1,yearmonth,yearmonth_p1]
    
    for month in seach_month:
        
            
        dirname = 'Z:/Landsat/wrs2/' + path_row + '/' + year + '/' + month + '/'
    
        print (dirname)
    
        pattern = '*_zstdmask.img'
    
        #imglist = fnmatch.filter(os.listdir(dirname), pattern)
    
        #print (imglist)
    
    